# **Quantium Chip Analysis**
### Quantium Retail Analytics Team: Customer Segmentation and Sales Drivers
### Analysis by: Breanna Williams
#### 02-12-2026

---

## **Overview:**
The following analysis has been requested by the Category Manager for chips to better understand the types of customers who purchase chips and their purchasing behaviour within the region. The insights from this analysis will feed into the supermarket’s strategic plan for the chip category in the next half year.

The data set is a sample from July 1, 2018 to Jun 30, 2019 and includes 26,4835 transactions from 72,637 unique customers, with 15 variables including: 

- 'date' 
- 'store_nbr' (store number)
- 'lylty_card_nbr' (loyalty card number)
- 'txn_id' (transaction ID)
- 'prod_nbr' (product number)
- 'prod_name' (product name)
- 'prod_qty' (product quantity)
- 'tot_sales' (total sales)
- 'prod_brand' (product brand)
- 'prod_size_grams' (product size in grams)
- 'prod_price' (product price)
- 'num_transactions' (number of transactions)
- 'sales_per_customer' (sales per customer)
- 'lifestage' (customer lifestage)
- 'premium_customer' (premium customer status)


### **Findings & Recommendations:**

1. Focus promotional and shelf space strategies on 150-190g chip packets priced between $3.00 and $3.99, as they have the highest total sales for all customer segments.

    ```python
    # There are 25 products matching these two requirements. Run this code to see the products that are between 150-190 grams and $3.00-3.99
    recommended_products = (data['prod_size_grams'] >= 150) & (data['prod_size_grams'] <= 190) & (data['prod_price'] >= 3.00) & (data['prod_price'] <= 3.99)
    data[recommended_products]['prod_name'].unique()
    ```

2. Additionally, maintain high visibility of Kettle brand chips with Doritos Corn Chips Supreme as a hero product, as they are top sellers across almost all lifestages.

    ```python
    # Use this code to see the top 5 selling Kettle products across all lifestages
    top_kettle = data.groupby(['prod_brand', 'prod_name'])['tot_sales'].sum().sort_values(ascending=False).reset_index()
    top_kettle[top_kettle['prod_brand'] == 'Kettle'].head(5)
    ```

3. Focus marketing efforts on older singles/couples, retirees, and older families as they make up over 50% of total sales, indicating that they are key customer segments for chips.

### **Specific Questions: Are chip packet sizes relative?**
Yes, there is a moderate positive association between product size and total sales (r=0.31), indicating that to some extent, larger product sizes are associated with higher total sales.

### **Customer Segments & Purchasing Behavior:**
All:
- Customers purchase products between 150-190 grams in size.
- Customers purchase products between $3.00-3.99.
- On average, customers spent $ 26.63 on chips within the span of the year.
- Kettle brand chips are the highest selling brand across all lifestages.
- Doritos Corn Chips Supreme sell the most across all lifestages except midage singles/couples whose top-selling chip product is Smiths Crinkle Chips.
- Mainstream customers have the highest total sales.

Young singles/couples:
- Customers in this group have the fewer chip transactions with the highest portion of the group making 3 transactions.
- Largest mainstream customer segment in terms of total sales.

Midage singles/couples:
- Customers in this group have a low to moderate number of chip transactions with the highest portion of the group making 6 transactions.

New Families:
- Represent the lowest number of chip transactions overall.
- Customers in this group have a low to moderate number of chip transactions with the highest portion of the group making 2 transactions.
- Spend the least amount of money on chips.

Older families:
- Make the most chip transactions.
- Largest Budget customer segment in terms of total sales.

Older singles/couples:
- Spend the most money on chips.
- Largest Premium customer segment in terms of total sales.

Retirees:
- Customers in this group have a low to moderate number of chip transactions with the highest portion of the group making 6 transactions.

Young families:
- Customers in this group have a moderate number of chip transactions with the highest portion of the group making 7 transactions. 

---
## **Full Analysis:**

### Import Libraries & Packages

In [3]:
# Initial imports
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Hypothesis testing imports
from scipy.stats import pearsonr

ModuleNotFoundError: No module named 'plotly'

### Import and Clean Transaction Data
---
We will use this section to import the transaction data and perform any necessary cleaning steps before analysis.

In [ ]:
# Loading transaction data
transaction_data = pd.read_csv('../data/QVI_transaction_data(in).csv')

In [ ]:
# Checking the structure of the transaction data
transaction_data.info()

In [ ]:
# Checking for duplicate rows in the transaction data
transaction_data.duplicated().sum()

In [ ]:
# Removing duplicates from the transaction data
transaction_data = transaction_data.drop_duplicates()

In [ ]:
# Changing column names to lowercase for ease of use
transaction_data.columns = transaction_data.columns.str.lower()

In [ ]:
# Converting excel date format to datetime
transaction_data['date'] = pd.to_datetime(transaction_data['date'], origin='1899-12-30', unit='D')

In [ ]:
# Verifying uniqueness between prod_name and prod_nbr columns to ensure there are no duplicate products with different names or ids
transaction_data['prod_name'].nunique() == transaction_data['prod_nbr'].nunique()

In [ ]:
# Using str.strip to remove any leading and trailing whitespace from product names
transaction_data['prod_name'] = transaction_data['prod_name'].str.strip()

In [ ]:
# Using str.split and str.join to remove extra spaces between words in the product names
transaction_data['prod_name'] = transaction_data['prod_name'].str.split().str.join(' ')

In [ ]:
# Replacing inconsistent brand names with the correct brand names so that they can be grouped together for analysis

ncc_replacements = {
    'Natural Chip Compny SeaSalt175g': 'Natural Chip Company Sea Salt 175g',
    'NCC Sour Cream & Garden Chives 175g': 'Natural Chip Company Sour Cream & Garden Chives 175g',
    'Natural ChipCo Hony Soy Chckn175g': 'Natural Chip Company Honey Soy Chicken 175g',
    'Natural Chip Co Tmato Hrb&Spce 175g': 'Natural Chip Company Tomato Herb & Spice 175g',
    'Natural ChipCo Sea Salt & Vinegr 175g': 'Natural Chip Company Sea Salt & Vinegar 175g'
}

for old, new in ncc_replacements.items():
    transaction_data['prod_name'] = transaction_data['prod_name'].str.replace(old, new, regex=False)

doritos_replacements = {
    'Dorito Corn Chp Supreme 380g': 'Doritos Corn Chips Supreme 380g',
    'Doritos Corn Chip Mexican Jalapeno 150g': 'Doritos Corn Chips Mexican Jalapeno 150g',
    'Doritos Corn Chip Southern Chicken 150g': 'Doritos Corn Chips Southern Chicken 150g'
 }

for old, new in doritos_replacements.items():
    transaction_data['prod_name'] = transaction_data['prod_name'].str.replace(old, new, regex=False)


transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('GrnWves', 'Grain Waves', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('Infzns', 'Infuzions', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('RRD', 'Red Rock Deli', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('Smith Crinkle Cut Mac N Cheese 150g', 'Smiths Crinkle Cut Mac N Cheese 150g', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('Smith Crinkle Cut Bolognese 150g', 'Smiths Crinkle Cut Bolognese 150g', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('Snbts Whlgrn', 'Sunbites Whlegrn', regex=False)

transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('WW', 'Woolworths',regex=False)



In [ ]:
# Extracting brand names from product names to create a new column for brand names
brands = ['Burger Rings', 'CCs', 'Cheetos', 'Cheezels', 'Cobs', 'Doritos', 'French Fries', 'Grain Waves', 'Infuzions', 'Kettle', 'Natural Chip Company', 'Old El Paso', 'Pringles', 'Red Rock Deli', 'Smiths', 'Sunbites', 'Thins', 'Tostitos', 'Twisties', 'Tyrrells','Woolworths']

def extract_brand(prod_name):
    for brand in brands:
        if prod_name.startswith(brand):
            return brand
    return 'Other'
    
transaction_data['prod_brand'] = transaction_data['prod_name'].apply(extract_brand)

In [ ]:
# Replacing the inconsistent product name with the correct name before we extract the product size
transaction_data['prod_name'] = transaction_data['prod_name'].str.replace('Kettle 135g Swt Pot Sea Salt', 'Kettle Swt Pot Sea Salt 135g')

In [ ]:
# Creating a new column for product size by extracting the last 4 characters from the product name
transaction_data['prod_size_grams'] = transaction_data['prod_name'].str[-4:-1].str.strip().str.lower().astype('int64')

In [ ]:
# Removing the product size from the product name to create a clean product name column for analysis
transaction_data['prod_name'] = transaction_data['prod_name'].str[:-4].str.strip()

#### Feature Engineering

In [ ]:
# Adding a column for product price to enable price analysis
transaction_data['prod_price'] = transaction_data['tot_sales'] / transaction_data['prod_qty']

In [ ]:
# Adding a column for num_transactions per customer to enable analysis of purchasing behaviour by number of transactions
transaction_data['num_transactions'] = transaction_data.groupby('lylty_card_nbr')['txn_id'].transform('nunique')

In [ ]:
# Adding a column for total sales per customer to enable analysis of purchasing behaviour by total sales
transaction_data['sales_per_customer'] = transaction_data.groupby('lylty_card_nbr')['tot_sales'].transform('sum')

#### Checking Outliers

In [ ]:
# Examining the summary statistics of the transaction data. There appear to be some outliers in prod_qty and tot_sales.
transaction_data.describe()

In [ ]:
# Function to identify outliers in the numeric columns of the transaction data using the IQR method
def num_outliers(df):
    ''' 
    Function to identify outliers in the numeric columns of the transaction data using the IQR method. 
    
    Parameters: (pandas dataframe)

    Returns: Prints the number of outliers in each numeric column of the transaction data
    '''
    outliers = []
    for col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - iqr * 1.5
        upper_bound = q3 + iqr * 1.5
        num_outliers = len(df[(df[col] < lower_bound) | (df[col] > upper_bound)])

        if num_outliers > 0:
            outliers.append({
                'column': col,
                'outliers': num_outliers
            })
    return pd.DataFrame(outliers)

In [ ]:
# Applying the num_outliers function to the numeric columns of the transaction data
outliers = num_outliers(transaction_data.select_dtypes(include=[np.number]))
outliers.plot(kind='bar', x='column')

In [ ]:
# Function to get the indices of the outliers in a specific column of the transaction data using the IQR method
def get_outlier_indices(df, col):
    '''
    Get the indices of the outliers in a specific column of the transaction data using the IQR method.

    Parameters:
    df (pandas DataFrame): The transaction data.
    col (str): The column name to check for outliers.

    Returns:
    pandas DataFrame: A DataFrame containing the outliers in the specified column.
    '''
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + (iqr * 1.5)
    lower_bound = q1 - (iqr * 1.5)

    return df[(df[col] < lower_bound) | (df[col] > upper_bound)]

In [ ]:
# Examining the outliers in the prod_size_grams column
get_outlier_indices(transaction_data, 'prod_size_grams')['prod_size_grams'].value_counts()

In [ ]:
# Examining the outliers in the prod_qty column
get_outlier_indices(transaction_data, 'prod_qty')['prod_qty'].value_counts()

In [ ]:
# Checking out the 2 outliers in the prod_qty column to see if they are valid data points or if they are errors that need to be removed from the analysis
transaction_data[transaction_data['prod_qty'] == 200]

We will leave the outliers in the data for now as they are legitimate.

---

### Import and Clean Customer Data
Examine customer data – check for similar issues in the customer data, look for nulls and when you are happy merge the transaction and customer data together so it’s ready for the analysis ensuring you save your files along the way.

In [ ]:
# Importing customer data
customer_data = pd.read_csv('../data/QVI_purchase_behaviour.csv')

In [ ]:
# Checking the structure of the customer data
customer_data.info()

In [ ]:
# Checking for duplicates in the customer data
customer_data.duplicated().sum()

In [ ]:
# Changing column names to lowercase for ease of use
customer_data.columns = customer_data.columns.str.lower()

In [ ]:
# Examining the value counts of the object columns in the customer data to check for inconsistencies
for col in customer_data.select_dtypes(include=['object']).columns:
    print(f"Unique values in column '{col}': {customer_data[col].value_counts()}")

In [ ]:
# Converting lifestage column to categorical data type 
customer_data['lifestage'] = customer_data['lifestage'].astype('category')

In [ ]:
# Converting premium_customer column to categorical data type and setting the order of the categories
customer_data['premium_customer'] = pd.Categorical(customer_data['premium_customer'], categories=['Budget', 'Mainstream', 'Premium'], ordered=True)

In [ ]:
# Converting lifestage column to lowercase
customer_data['lifestage'] = customer_data['lifestage'].str.lower()

In [ ]:
# Converting premium_customer column to lowercase
customer_data['premium_customer'] = customer_data['premium_customer'].str.lower()

#### Merging Data

In [ ]:
# Merging the datasets
data = transaction_data.merge(customer_data, on='lylty_card_nbr')

In [ ]:
# Examining the structure of the merged dataset
data.info()

---
### **Exploratory Data Analysis:**
In this section we will explore the drivers of sales for chips. We will look at total sales, where the highest sales are coming from, and any interesting trends or insights we can find. We will also look at average sales per customer and how it varies across different customer segments.

#### Correlation Heat Map
Using a correlation heat map to gain an initial understanding of relationships between numeric variables in the dataset. However, some of the raw variables don't tell us much, so we will aggregate the data to gain better insights.

In [ ]:
# Plotting the correlation heatmap to understand relationships between numeric variables.
sns.heatmap(data.corr(numeric_only=True).round(2), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Numeric Variables')   
plt.show()

---
#### Total Sales by Product Size: Does Packet Size Impact Overall Sales?
Quick Answer: Yes, there is a moderate positive association between product size and total sales.

In [ ]:
# Plotting a histogram to show total sales by product size, colored by lifestage
lifestage_colors = {
    'older singles/couples': "#19BAFF",
    'retirees': "#EB2ADE",
    'older families': "#6D01F1",
    'young families': "#FFEE31",
    'young singles/couples': "#009F82",
    'midage singles/couples': "#FF000D",
    'new families': "#FF8D46"
}

px.histogram(data,
             x='prod_size_grams', 
             y='tot_sales', 
             nbins=10, 
             histfunc='sum', 
             title='Total Sales by Product Size', 
             labels={'prod_size_grams': 'Product Size (grams)', 'tot_sales': 'Total Sales'}, 
             color='lifestage', 
             color_discrete_map=lifestage_colors)

##### Key insights:
**Products between 150-190 grams have the highest total sales for all customer segments.**

- Total sales and product size have a moderate positive association (r=0.31), indicating that to some extent, larger product sizes are associated with higher total sales.
- The distribution of total sales by product size is right-skewed indicating that *most sales are concentrated in smaller product sizes*.
- Products less than 100 grams sold the least.

---

#### Total Sales by Product Price: Does Product Price Impact Overall Sales?
Quick Answer: Yes, there is a moderate positive association between product price and total sales.

In [ ]:
# Grouping the data by product price to create a table for the scatter plot
sales_by_price = data.groupby('prod_price').agg({'tot_sales': 'sum'}).round(2).reset_index()

In [ ]:
# Checking the correlation of the aggregated sales_by_price table to see if there is a correlation between product price and total sales
sales_by_price.corr(numeric_only=True).round(2)

In [ ]:
# Creating a scatter plot to show total sales by product price with a trendline to see if there is a relationship between product price and total sales
scatter1 = px.scatter(
    data_frame=sales_by_price,
    x='prod_price',
    y='tot_sales',
    title='Total Sales by Product Price',
    labels={'prod_price': 'Product Price', 'tot_sales': 'Total Sales'},
    trendline='ols',
    trendline_color_override='red' # Ensures the trendline displays
)

# Creating a histogram to show total sales by product price, colored by lifestage
histogram1 = px.histogram(
    data_frame=data,
    x='prod_price',
    y='tot_sales',
    title='Total Sales by Product Price',
    labels={'prod_price': 'Product Price', 'tot_sales': 'Total Sales'},
    color='lifestage',
    color_discrete_map=lifestage_colors,
    nbins=10,
    histfunc='sum'
)

In [ ]:
# Creating a subplot with a scatter plot and a histogram stacked vertically
fig1 = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.15,
    subplot_titles=("Scatter Plot", 'Distribution'),
    row_heights=[0.7, 0.7],
    specs=[[{'type': 'scatter'}], [{'type': 'histogram'}]])

# Adding the scatter traces to the first row of the subplot
for trace in scatter1.data:
    fig1.add_trace(trace, row=1, col=1)

# Adding the histogram traces to the second row of the subplot
for trace in histogram1.data:
    fig1.add_trace(trace, row=2, col=1)

# Adding title and adjusting size of the figure
fig1.update_layout(
    title='Total Sales by Product Price',
    height=800,
    width=800)

fig1.show()

##### Key Insights:
**Products between $3 and $3.99 have the highest total sales across all lifestages**.

- There is a moderate positive association (r=0.43) between total sales and product price, indicating that higher-priced products tend to have higher sales overall.

---

#### Total Sales by Lifestage: Does Customer Lifestage Impact Overall Sales?
Quick Answer: Yes, customers in the older segments make up over half of the total sales.

In [ ]:
# Creating a pie chart to show total sales by lifestage to see if there are differences in total sales by customer lifestage 
px.pie(data, 
       names='lifestage', 
       values='tot_sales', 
       labels={'index': 'Lifestage', 'value': 'Total Sales'}, 
       title='Total Sales by Lifestage', 
       color='lifestage',
       color_discrete_map=lifestage_colors)

##### Key Insights:
**Customers in the older segments make up over 50% of total sales, indicating that they are a key customer segment for chips.**

- Older singles/couples are the largest customer segment in terms of total sales followed by retirees and older families. 
- New families are the smallest customer segment in terms of total sales, making up only 2.6% of total sales.

---

#### Number of Transactions per Customer - Does the Number of Transactions per Customer Affect Sales? Which Customer Segments have the Highest Number of Transactions?
Quick Answer: Yes, customers with more transactions tend to have higher total sales. Older families have the highest number of transactions, followed by young families.

In [ ]:
# Creating a histogram to show the distribution of number of transactions per customer by lifestage
px.histogram(data, 
             x='num_transactions',
             title='Distribution of Transactions per Customer', 
             color='lifestage',
             labels={'num_transactions': 'Number of Transactions', 'num_customers': 'Number of Customers'},
             color_discrete_map=lifestage_colors, 
             nbins=20, 
             barmode='group')

In [ ]:
# Creating a pie chart to show the proportion of transactions per customer by lifestage
px.pie(data,
       names='lifestage',
       values='num_transactions',
       title='Pie Chart of Transactions by Lifestage',
       color='lifestage', 
       color_discrete_map=lifestage_colors)

##### Key Insights:
**Both young and older families have a higher number of transactions compared to the other lifestages with a higher concentration of transactions at 7 transactions.**
**Older families represent the highest number of transactions.**

- Overall the distribution of total transactions per customer is right-skewed, indicating that most customers have a low-moderate number of chip transactions, while a small number of customers have a higher number of chip transactions.
- Older families tend to have have a higher number of transactions with the highest concentration of transactions at 7 transactions.
- New families represent the lowest number of transactions overall with the highest concentration of transactions at 2 transactions.

---

#### Total Sales by Brand and Product: Which Brands or Products Most Impact Overall Sales?
Quick Answer: Kettle brand chips are the highest selling brand across all lifestages and Doritos Corn Chips Supreme sell the most across all lifestages except midage singles/couples whose top-selling chip product is Smiths Crinkle Chips.

In [ ]:
# Grouping the data by product name and product brand to show total sales and total quantity sold by product
sales_by_brand = data.groupby(['prod_brand', 'lifestage']).agg({'tot_sales': 'sum'}).reset_index().sort_values(by=['tot_sales'], ascending=[False])


In [ ]:
# Grouping the data again by lifestage and showing the top 5 brands for each lifestage based on total sales
sales_by_brand_grouped = sales_by_brand.groupby('lifestage').head(5)

In [ ]:
# Grouping the data by product name to show total sales and total quantity sold by product
sales_by_product = data.groupby(['prod_name', 'lifestage']).agg({'tot_sales': 'sum'}).reset_index().sort_values(by=['tot_sales'], ascending=[False])

In [ ]:
# Grouping the data again by lifestage and showing the top 5 products for each lifestage based on total sales
sales_by_product_grouped = sales_by_product.groupby('lifestage').head(5)

In [ ]:
# Creating the histogram for total sales by brand and product, colored by lifestage

lifestage_order = ['older singles/couples', 'older families', 'retirees', 
                   'young families', 'young singles/couples', 
                   'midage singles/couples', 'new families']        # Ensuring lifestage categories are in a logical order for both dropdown menus

# Creating the histogram for total sales by brand, colored by lifestage
histogram5 = px.histogram(sales_by_brand_grouped, 
             x='prod_brand', 
             y='tot_sales', 
             labels={'prod_brand': 'Brand Name', 'tot_sales': 'Total Sales'}, title='Total Sales by Brand', 
             color='lifestage',
             color_discrete_map=lifestage_colors,
             category_orders={'lifestage': lifestage_order})

# Creating the histogram for total sales by product, colored by lifestage
histogram6 = px.histogram(sales_by_product_grouped, 
             x='prod_name', 
             y='tot_sales', 
             labels={'prod_name': 'Product Name', 'tot_sales': 'Total Sales'}, title='Total Sales by Product', 
             color='lifestage',
             color_discrete_map=lifestage_colors,
             category_orders={'lifestage': lifestage_order})

In [ ]:
# Adding a dropdown menu to filter the data by lifestage. Traces 0-6 are brand data and traces 7-13 are product data.

brand_dropdown = [
    {'label': 'All',
    'method': 'update',
    'args': [{'visible': [True]*7 + [False]*7}]},
             
    {'label': 'Older Singles/Couples',
    'method': 'update',
    'args': [{'visible': [True] + [False]*6 + [False]*7}]},

    {'label': 'Older Families',
    'method': 'update',
    'args': [{'visible': [False, True] + [False]*12}]},
        
    {'label': 'Retirees',
    'method': 'update',
    'args': [{'visible': [False]*2 + [True] + [False]*11}]},

    {'label': 'Young Families',
    'method': 'update',
    'args': [{'visible': [False]*3 + [True] + [False]*10}]},

    {'label': 'Young Singles/Couples',
    'method': 'update',
    'args': [{'visible': [False]*4 + [True] + [False]*9}]},

    {'label': 'Midage Singles/Couples',
    'method': 'update',
    'args': [{'visible': [False]*5 + [True] + [False]*8}]},

    {'label': 'New Families',
    'method': 'update',
    'args': [{'visible': [False]*6 + [True] + [False]*7}]}
 ]

product_dropdown = [
    {'label': 'All',
    'method': 'update',
    'args': [{'visible': [False]*7 + [True]*7}]},
             
    {'label': 'Older Singles/Couples',
    'method': 'update',
    'args': [{'visible': [False]*7 + [True] + [False]*6}]},

    {'label': 'Older Families',
    'method': 'update',
    'args': [{'visible': [False]*8 + [True] + [False]*5}]},
        
    {'label': 'Retirees',
    'method': 'update',
    'args': [{'visible': [False]*9 + [True] + [False]*4}]},

    {'label': 'Young Families',
    'method': 'update',
    'args': [{'visible': [False]*10 + [True] + [False]*3}]},

    {'label': 'Young Singles/Couples',
    'method': 'update',
    'args': [{'visible': [False]*11 + [True] + [False]*2}]},

    {'label': 'Midage Singles/Couples',
    'method': 'update',
    'args': [{'visible': [False]*12 + [True, False]}]},

    {'label': 'New Families',
    'method': 'update',
    'args': [{'visible': [False]*13 + [True]}]}]

# Adding buttons to toggle between product and brand data. Traces 0-6 are product data and traces 7-13 are brand data.
buttons = [
    {'label': 'Total Sales by Brand',
     'method': 'update',
     'args': [{'visible': [True]*7 + [False]*7},
              {'title': 'Total Sales by Product/Brand - Brand',
               'updatemenus[0].visible': True,                      # Show the brand dropdown menu - set in update layout
               'updatemenus[1].visible': False}]},
               
    {'label': 'Total Sales by Product',
     'method': 'update',
     'args': [{'visible': [False]*7 + [True]*7},
              {'title': 'Total Sales by Product/Brand - Product',
               'updatemenus[0].visible': False,
               'updatemenus[1].visible': True}]}
]


In [ ]:
# Creating the figure and adding the traces from both histograms to the figure. The dropdown menu will filter the data by lifestage and the buttons will toggle between product and brand data.
fig4 = go.Figure()

# Adding brand histogram
for trace in histogram5.data:
    fig4.add_trace(trace)

# Adding product histogram
for trace in histogram6.data:
    trace.showlegend = False    # Removing duplicate legend for product data
    trace.visible = False       # Setting default visibility for product data
    fig4.add_trace(trace)

# Adding title and dropdown menu to filter by lifestage and buttons to toggle between product and brand data
fig4.update_layout({
    'title': 'Total Sales by Product/Brand',
    'updatemenus': [{'type': 'dropdown',
                     'buttons': brand_dropdown,
                     'direction': 'down',   
                     'showactive': True,
                     'x': 1.3,
                     'y': 1.2,
                     'visible': True},

                     {'type': 'dropdown',
                     'buttons': product_dropdown,
                     'direction': 'down',   
                     'showactive': True,
                     'x': 1.3,
                     'y': 1.2,
                     'visible': False},

                     {'type': 'buttons',
                    'buttons': buttons,
                    'direction': 'right',  
                    'x': 0.98,
                    'y': 1.25}]
                    })

fig4.show()


##### Key Insights:
**Kettle brand chips are the highest selling brand across all lifestages.**
**Doritos Corn Chips Supreme sell the most across all lifestages except midage singles/couples whose top-selling chip product is Smiths Crinkle Chips.**

- While Kettle has the strongest portfolio of products with the most products in the top 10 best selling products across all lifestages, they do not have the single best selling product across all lifestages. This indicates that while Kettle has a strong overall brand presence, they may not have a standout product that is driving sales across all lifestages.

---

#### Total Sales by Customer Type: Do Premium Customers Spend More?
Quick Answer: No, mainstream customers have the highest total sales.

In [ ]:
# Creating a pie chart to show total sales by customer type
px.pie(data, 
       names='premium_customer', 
       values='tot_sales', 
       title='Total Sales by Customer Type', 
       labels={'premium_customer': 'Customer Type', 'tot_sales': 'Total Sales'},
       category_orders={'premium_customer': ['budget', 'mainstream', 'premium']})

In [ ]:
# Creating a histogram to show total sales by customer type, colored by lifestage
px.histogram(
    data,
    x='premium_customer',
    y='tot_sales',
    color='lifestage',
    color_discrete_map=lifestage_colors,
    category_orders={'premium_customer': ['budget', 'mainstream', 'premium']},
    title='Total Sales by Customer Type and Lifestage',
    labels={'premium_customer': 'Customer Type', 'tot_sales': 'Total Sales'}
)

##### Key Insights:
**Mainstream customers have the highest total sales.**

- Older families are the largest budget customers in terms of total sales.
- Young singles/couples are the largest mainstream customers in terms of total sales.
- Older singles/couples are the largest premium customers in terms of total sales.

---

## **Conclusion:**

In conclusion, the analysis of the chip market has revealed several key insights that can inform strategic decisions for the supermarket. The findings indicate that product size and price are important drivers of sales, with products between 150-190 grams and priced between $3.00-3.99 performing the best across all customer segments. 

Additionally, older customers, particularly older singles/couples, retirees, and older families make up a significant portion of total sales, suggesting that marketing efforts should be targeted towards these segments. 

Kettle brand chips have a strong presence in the market, but there may be an opportunity to develop standout products that can drive sales across all lifestages and Doritos Corn Chips Supreme is a strong performing product across all lifestages except midage singles/couples where Smiths Crinkle Chips is the top selling product.

Overall, these insights can help the supermarket optimize its product offerings and marketing strategies to better meet the needs of its customers and drive sales in the chip category.